# Package Calling

In [1]:
import sys

sys.path.append("../../")

In [3]:
import datetime as dtt
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## Package Settings

In [4]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [5]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [6]:
plt.rcParams["font.family"] = ["Arial Unicode MS"]  # Chinese Labels
plt.rcParams["axes.unicode_minus"] = False  # Minus Sign

sns.set(
    style="darkgrid",
    rc={
        "figure.figsize": (20, 8),
        "font.sans-serif": ["Arial Unicode MS", "Arial"],
    },
)

## GC Settings

In [7]:
import gc

gc.isenabled()
gc.get_threshold()

True

(700, 10, 10)

In [8]:
# gc.set_threshold(10,1,1)
# gc.enable()
# gc.disable()

# Modeling

In [9]:
data_path = "/Users/chenzhou/Documents/Everything/python/COVID19/data/data_0516_SH.csv"
data_sh = pd.read_csv(
    data_path,
    dtype={
        "inbound_confirmed": int,
        "inbound_asymp": int,
        "outbound_confirmed": int,
        "outbound_asymp": int,
        "cure": int,
        "relieve": int,
    },
    parse_dates=["date"],
)

In [10]:
data_sh["confirmed_add"] = data_sh.inbound_confirmed + data_sh.outbound_confirmed
data_sh["asymp_add"] = data_sh.inbound_asymp + data_sh.outbound_asymp
data_sh["total_add"] = data_sh.confirmed_add + data_sh.asymp_add

data_sh["confirmed_acc"] = data_sh.confirmed_add.cumsum() - 380
data_sh["asymp_acc"] = data_sh.asymp_add.cumsum() - 120
data_sh["total_affected"] = data_sh.confirmed_acc + data_sh.asymp_acc

data_sh = data_sh.loc[data_sh.date >= "2022-03-01"].reset_index(drop=True)

In [11]:
data_sh

,inbound_confirmed,inbound_asymp,outbound_confirmed,outbound_asymp,date,cure,relieve,confirmed_add,asymp_add,total_add,confirmed_acc,asymp_acc,total_affected
0,1,1,37,17,2022-03-01,8,1,38,18,56,95,5,100
1,3,5,39,19,2022-03-02,8,2,42,24,66,137,29,166
2,2,14,43,21,2022-03-03,9,1,45,35,80,182,64,246
3,3,16,24,10,2022-03-04,12,0,27,26,53,209,90,299
4,0,28,25,10,2022-03-05,8,4,25,38,63,234,128,362
5,3,45,32,16,2022-03-06,65,16,35,61,96,269,189,458
6,4,51,36,10,2022-03-07,27,10,40,61,101,309,250,559
7,3,62,26,10,2022-03-08,32,7,29,72,101,338,322,660
8,4,76,42,16,2022-03-09,18,13,46,92,138,384,414,798
9,11,64,32,10,2022-03-10,17,7,43,74,117,427,488,915


## Bezier Curve

In [12]:
from skfda.representation.basis import BSpline

In [192]:
bs_interp_prop = 0.5
interg_interp_prop = 500.0
ser = data_sh.total_affected.iloc[:28].values
x = range(len(ser))
kappa, mu, tau = 0.5, 1.60, 1.17

In [193]:
labd, lr, decay, iters, show_process, iter_opt = 5.0, 0.1, 0.99, int(1e2), True, False

In [194]:
knots = np.linspace(0, len(ser) - 1, round(bs_interp_prop * (len(ser) - 1) + 1))
integ_x = np.linspace(0, len(ser) - 1, round(interg_interp_prop * (len(ser) - 1) + 1))

In [195]:
bs_func = BSpline(domain_range=(min(x), max(x)), order=4, knots=knots)
bs_func_d = bs_func.derivative()

In [196]:
bS = bs_func(x).squeeze(2)
bs_integ = bs_func(integ_x).squeeze(2)
bs_partial = bs_func_d(integ_x).squeeze(2)

In [197]:
shift_ = round(tau * interg_interp_prop)
now_bs_integ = bs_integ[:, shift_:]
delay_bs_integ = bs_integ[:, :-shift_]
now_bs_partial = bs_partial[:, shift_:]
delay_bs_partial = bs_partial[:, :-shift_]

In [198]:
n_coeff = bS.shape[0]
init_coeff = np.repeat(0, n_coeff)
init_grad = np.repeat(0, n_coeff)

In [199]:
loglik_loss = np.linalg.norm(loglik_loss_vec, 2)

In [200]:
for _iter in range(iters):
    # loss calculation
    loglik_loss_vec = np.dot(init_coeff, bS) - ser
    loglik_loss = np.linalg.norm(loglik_loss_vec, 2)
    mat_ = now_bs_partial - kappa * now_bs_integ + kappa * mu * delay_bs_integ
    integ_loss_vec = np.dot(init_coeff, mat_)
    integ_loss = np.linalg.norm(integ_loss_vec, 2) / np.sqrt(interg_interp_prop) * labd
    loss_ = loglik_loss + integ_loss

    # gradient descent
    loglik_part = np.dot(bS, loglik_loss_vec)
    integ_part = np.dot(mat_, np.dot(init_coeff, mat_)) / interg_interp_prop * labd**2
    init_coeff = init_coeff - lr * (loglik_part + integ_part)
    break

In [225]:
coef_mat = np.dot(bS, bS.T) + np.dot(mat_, mat_.T) / interg_interp_prop * labd**2
k_mat = np.dot(
    np.dot(
        now_bs_integ - mu * delay_bs_integ,
        (now_bs_integ - mu * delay_bs_integ - now_bs_partial).T,
    ),
    init_coeff,
)
m_mat = np.dot(
    (mu * kappa**2) * np.dot(delay_bs_integ, delay_bs_integ.T)
    + kappa * np.dot(delay_bs_integ, (now_bs_partial - kappa * now_bs_integ).T),
    init_coeff,
)
t_mat = -np.dot(
    (
        2
        * kappa
        * mu
        * np.dot(
            now_bs_partial - kappa * now_bs_integ + kappa * mu * delay_bs_integ,
            delay_bs_partial.T,
        )
        + (now_bs_partial - kappa * now_bs_integ + kappa * mu * delay_bs_integ)[:, [0]]
        ** 2
    ),
    init_coeff,
)

In [236]:
partial_resid_theta = np.dot(np.dot(bS.T, np.linalg.inv(coef_mat)), np.array([k_mat,m_mat,t_mat]).T) * 2 * labd**2 / interg_interp_prop

In [238]:
theta_grad = np.dot(np.dot(np.linalg.inv(np.dot(partial_resid_theta.T, partial_resid_theta)), partial_resid_theta.T),  ser - np.dot(bS.T, init_coeff))

In [239]:
theta_grad

array([-0.52067459,  4.3051547 ,  5.29120492])

In [ ]:
sns.lineplot(x=range())